### **Extraccion Full de "Crypto Aggregates API" de https://finage.co.uk/**
En este caso, ya que **"Crypto Aggregates API"** dará acceso a todos los tick (Un tick representa la variación entre dos precios consecutivos) de las criptomonedas disponibles con una solicitud de API, y la finalidad de esta extracción es solo obtener los últimos ticks de una determinada criptomoneda (es decir, no hay intencion de realizar un registro historico), por lo que se decidió utilizar el Metodo de Extracción de Tipo Full.

In [1]:
#Importo todas las Librerias que utilizaré

import pandas as pd
from datetime import datetime, timedelta
from configparser import ConfigParser
from utils_API import *
from utils_state import *
from utils_parquet import *

In [2]:
#Establezco como nueva fecha inicial la fecha de hace tres dias 
#atras del dia actual, y como fecha final la fecha del dia actual
full_start_date = datetime.utcnow() - timedelta(days=3)
full_start_date = full_start_date.strftime("2020-%m-%d")
full_end_date = datetime.utcnow()
full_end_date = full_end_date.strftime("2020-%m-%d")

#Impresion de control
print(full_start_date, type(full_start_date))
print(full_end_date, type(full_end_date))

2020-04-08 <class 'str'>
2020-04-11 <class 'str'>


In [3]:
#Instancio ConfigParser() de configparser en la variable parser y con el leo el archivo de configuracion pipeline.conf
parser = ConfigParser()
parser.read('pipeline.conf')

#Extraigo la informacion de la seccion 'finage' de pipeline.conf
api_credentials = parser["finage"]

#Guardo la informacion extraida en sus variables correspondientes
api_key = api_credentials["api_key"]
limit = api_credentials["limit"] #El limite sera de 3 debido a que la API me permite solo 1.000 Requests por mes

In [4]:
#Establezco la url base y los parametros con la información extraida 
#de pipeline.conf para realizar la query segun la documentacion de la API
base_url = "https://api.finage.co.uk"
params = {'limit':limit, 'sort':'asc', 'dbt_filter':'false', 'st':'11:00', 'et':'16:00', 'apikey':api_key}

In [5]:
#Creo una lista con los simbolos de las criptomonedas, con montos 
#expresados en dolares, de las que solicitare los ticks
symbols = ['BTCUSD', 'ETHUSD', 'ADAUSD', 'DOGEUSD']

#Creo una lista donde guardare los dataframes con los ticks de cada criptomoneda
df_crypto_ticks = []

for symbol in symbols:
    #Modifico el endpoint para obtener informacion de los ticks de cada criptomoneda en la lista symbols
    endpoint = f"agg/crypto/{symbol}/{'1'}/{'day'}/{full_start_date}/{full_end_date}"

    #hago la request con get_data() de utils_API.py
    json_data = get_data(base_url, endpoint, params=params)

    #Construyo un DataFrame con build_table() de utils_API.py usando la lista de 
    #objetos dentro de la clave 'results' del JSON devuelto por la API
    df_current_crypto_ticks = build_table(json_data, 'results')

    #Agrego la columna 'symbol' al DataFrame
    df_current_crypto_ticks['symbol'] = symbol

    #Reordeno las columnas del DataFrame
    columns = ['symbol', 'o', 'c', 'h', 'l', 'v', 't']

    #Reindexo el DataFrame con las columnas en el nuevo orden para volverlo mas intuitivo
    df_current_crypto_ticks = df_current_crypto_ticks.reindex(columns=columns)

    #Impresion de control
    print(df_current_crypto_ticks)

    #Agrego el dataframe de los ticks de la criptomoneda actual a la lista de dataframes df_crypto_ticks
    df_crypto_ticks.append(df_current_crypto_ticks)

   symbol        o        c        h        l              v              t
0  BTCUSD  7197.32  7361.28  7420.00  7150.00   76059.145838  1586390399999
1  BTCUSD  7360.26  7283.54  7371.92  7108.08   61094.872417  1586476799999
2  BTCUSD  7283.54  6858.92  7295.75  6739.98  104674.623375  1586563199999
   symbol       o       c       h       l             v              t
0  ETHUSD  164.62  173.11  174.48  163.59  1.023576e+06  1586390399999
1  ETHUSD  173.15  169.52  173.38  165.19  7.198372e+05  1586476799999
2  ETHUSD  169.56  157.81  170.10  152.11  1.314841e+06  1586563199999
   symbol        o        c        h        l            v              t
0  ADAUSD  0.03528  0.03639  0.03683  0.03522  220316016.7  1586390399999
1  ADAUSD  0.03638  0.03607  0.03660  0.03518  158350204.8  1586476799999
2  ADAUSD  0.03609  0.03321  0.03620  0.03215  297588582.1  1586563199999
    symbol         o         c         h         l          v              t
0  DOGEUSD  0.001997  0.002004  0.00204

In [7]:
#Construyo un dataframe con todos los dataframes de los ticks de cada criptomoneda 
#almacenados en df_crypto_ticks con el metodo 'concat()' de pandas
df_full = pd.concat(df_crypto_ticks, ignore_index=True)
df_full.head(12)

,symbol,o,c,h,l,v,t
0,BTCUSD,7197.320000,7361.280000,7420.000000,7150.000000,7.605915e+04,1586390399999
1,BTCUSD,7360.260000,7283.540000,7371.920000,7108.080000,6.109487e+04,1586476799999
2,BTCUSD,7283.540000,6858.920000,7295.750000,6739.980000,1.046746e+05,1586563199999
3,ETHUSD,164.620000,173.110000,174.480000,163.590000,1.023576e+06,1586390399999
4,ETHUSD,173.150000,169.520000,173.380000,165.190000,7.198372e+05,1586476799999
5,ETHUSD,169.560000,157.810000,170.100000,152.110000,1.314841e+06,1586563199999
6,ADAUSD,0.035280,0.036390,0.036830,0.035220,2.203160e+08,1586390399999
7,ADAUSD,0.036380,0.036070,0.036600,0.035180,1.583502e+08,1586476799999
8,ADAUSD,0.036090,0.033210,0.036200,0.032150,2.975886e+08,1586563199999
9,DOGEUSD,0.001997,0.002004,0.002043,0.001967,9.038491e+07,1586390399999


In [8]:
#Convierto la columna 't' de timestamp a tipo datetime (se necesita agregar unit='ms' porque los timestamp son de tipo numpy.int64)
df_full['t'] = pd.to_datetime(df_full['t'], unit='ms')

#Formateo los timestamps en la columna como fechas con el formato 'YYYY-mm-dd'
df_full['t'] = df_full['t'].dt.strftime('%Y-%m-%d')

#Imprimo el nuevo DataFrame
df_full.head(12)

,symbol,o,c,h,l,v,t
0,BTCUSD,7197.320000,7361.280000,7420.000000,7150.000000,7.605915e+04,2020-04-08
1,BTCUSD,7360.260000,7283.540000,7371.920000,7108.080000,6.109487e+04,2020-04-09
2,BTCUSD,7283.540000,6858.920000,7295.750000,6739.980000,1.046746e+05,2020-04-10
3,ETHUSD,164.620000,173.110000,174.480000,163.590000,1.023576e+06,2020-04-08
4,ETHUSD,173.150000,169.520000,173.380000,165.190000,7.198372e+05,2020-04-09
5,ETHUSD,169.560000,157.810000,170.100000,152.110000,1.314841e+06,2020-04-10
6,ADAUSD,0.035280,0.036390,0.036830,0.035220,2.203160e+08,2020-04-08
7,ADAUSD,0.036380,0.036070,0.036600,0.035180,1.583502e+08,2020-04-09
8,ADAUSD,0.036090,0.033210,0.036200,0.032150,2.975886e+08,2020-04-10
9,DOGEUSD,0.001997,0.002004,0.002043,0.001967,9.038491e+07,2020-04-08


### **Almacenamiento de los DataFrames en un Data Lake en formato Parquet**

In [9]:
#Creo la ruta en donde se almacenaran los archivos parquet
bronze_dir = "datalake/bronze/finage_api"

In [10]:
#Guardo el DataFrame de la extraccion full como archivo parquet con save_to_parquet() de utils_parquet.py 
#en el directorio Crypto_Aggregates de finage_api de la capa bronze del Data Lake 
save_to_parquet(
    df_full,
    f"{bronze_dir}/Crypto_Aggregates/data.parquet"
    )